In [1]:
from gurobipy import *
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

In [10]:
# Create constants
dt = 0.1
M = 100
o = np.random.randn(M,1)
og = np.random.randn(M,1)
B = np.random.randn(M,M)
C = np.random.randn(M,2)
D = np.eye(M)

In [11]:
m = Model("model")
m.setParam('OutputFlag', 0) 

u1 = m.addVar(name="u1", lb=-GRB.INFINITY, ub=GRB.INFINITY)
u2 = m.addVar(name="u2", lb=-GRB.INFINITY, ub=GRB.INFINITY)

u = np.array([[u1], [u2]])
distance = (og - (o + np.dot(B, o) + dt*np.dot(C, u)))
obj = np.dot(np.dot(distance.T, D), distance)[0, 0]
m.setObjective(obj, GRB.MINIMIZE)
M = 1
m.addQConstr(u1*u1 + u2*u2 <= M**2, "c0")

m.optimize()

In [12]:
def obj_func(params):
    u = params.reshape(2,1)
    distance = (og - (o + np.dot(B, o) + dt*np.dot(C, u)))
    obj = np.dot(np.dot(distance.T, D), distance)
    return obj
    
max_speed_constraint =  {'type': 'ineq', 'fun': lambda u: M - np.linalg.norm(u)}
result = optimize.minimize(obj_func, x0=np.array([[0], [0]]), constraints=max_speed_constraint)
if result.success:
    print(result.x)
else:
    print(result)

[ 0.8909477   0.45410593]


# Which is faster? Gurobi or Scipy?

In [13]:
%%timeit -n10000
# Gurobi
m.optimize()

10000 loops, best of 3: 132 µs per loop


In [14]:
%%timeit -n10000
# Scipy
optimize.minimize(obj_func, x0=np.array([[0], [0]]), constraints=max_speed_constraint)

10000 loops, best of 3: 1.32 ms per loop
